In [1]:
# ## IMPORTS

# import os
# import sys
# import time
# import math
# from datetime import datetime
# from io import StringIO


# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import plotly.graph_objects as go
# from numpy.polynomial.polynomial import Polynomial
# import scipy.integrate as integrate
# import scipy.optimize as sco
# from scipy.optimize import minimize
# from sklearn.metrics import mean_squared_error
# import openpyxl
# import networkx as nx
# import sklearn.metrics
# from IPython.core.display import HTML
# import plotly.io as pio
# from openpyxl.utils.dataframe import dataframe_to_rows
# import openpyxl as opxl
# import matplotlib.ticker as mtick
# from matplotlib.cm import get_cmap
# import time


# sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\parallel-flow-distribution-pressure-loss\ansys')
# sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\PVT-thermal-performance-model')
# import jou_gen as jg
# import ansys_py_bridge as apb
# import ansys.fluent.core as pyfluent
# # import model as ty
# import proc as pr
# import plot_functions_here as pfun
# import heat_transfer as bht
# import fluids as fds
# import ht
# import general as gen

In [2]:
## IMPORTS
import os
import sys
import re
import time
import math
from datetime import datetime
from io import StringIO

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.colors as pc
from scipy.stats import linregress
from numpy.polynomial.polynomial import Polynomial
import scipy.integrate as integrate
import scipy.optimize as sco
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import openpyxl
import networkx as nx
import sklearn.metrics
from IPython.core.display import HTML
import plotly.io as pio
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl as opxl
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
import time
import pickle

import bridge_plot as bplt

In [3]:
SR_fp = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\SimulationResults'
caoMeshCode = 'A5_M1'
testConditionsCode = 'tC0'
method = 'bridge'
no_try = 0
no_case = 0

In [45]:
(panelSpecs, hyp, stepCondition, log, CFD_ht_list, phis_list, df_one_per_part_list) = bplt.get_data_v2(SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case)

In [ ]:
df_one_per_part_list[0]['Qdot_PV_sky'] 

In [ ]:
Qdot = 'Qdot_PV_sky'
bplt.plot_1D_last_it(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()
bplt.plot_CFD_tot_last_it(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()

In [ ]:
Qdot = "Qdot_PV_sky"
bplt.plot_big_it(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()

In [ ]:
caoMeshCode = 'A12_M1'
bplt.plot_1D_DeltaT_v2('Qdot_tube_fluid', SR_fp, caoMeshCode, testConditionsCode, method, no_try, [0,1,2])

In [ ]:
print(23.61/1.95)
print(25.4/1.95)
print(25.64/1.95)
print(29.70/1.95) # A8
print(28.8/1.95) # A10_M2
print(28.7/1.95) # A11_M1
print(29.77/1.95)

In [ ]:
(13.02-12.107) / 12.107

In [ ]:
caoMeshCode_list = ['A3_M1','A4_M1','A5_M1', 'A7_M1', 'A8_M1']
# no_case = 1
no_try = 0
for no_case in range(0, 3):
    bplt.plot_1D_participation_stacked_last_it_multiple(SR_fp, caoMeshCode_list, testConditionsCode, method, no_try, no_case).show()

In [ ]:
panelSpecs, hyp, stepCondition, log, CFD_ht_list, phis_list, df_one_per_part_list = bplt.get_data_v2(SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case)
stepCondition

In [ ]:
# SR_fp = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel\CFD\Fluent_GMI\SimulationResults'
SR_fp = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\SimulationResults'
# caoMeshCode = 'A5_M1'
caoMeshCode_list = ['A5_M1', 'A7_M1']
testConditionsCode = 'tCwind'
method = 'bridge'
no_try =0
no_case = 0
cases = [0,1,2,3]
df_list_1 = []
df_list_2 = []
Qdot = 'Qdot_tube_fluid'
# bplt.plot_1D_participation_stacked_last_it_multiple(SR_fp, caoMeshCode_list, testConditionsCode, method, no_try, no_case).show()
for caoMeshCode in caoMeshCode_list :
    cases = [0, 1, 2, 3]
    bplt.plot_1D_participation_stacked_last_it_multiple_cases(SR_fp, caoMeshCode, testConditionsCode, method, no_try, cases).show()
    # bplt.plot_1D_DeltaT_v2(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, cases).show()
    # df = bplt.generate_regression_table(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, cases)
    # df = pd.DataFrame(df, index=[0])
    # df_list.append(df)
    cases = [0, 1]
    df_1 = bplt.generate_regression_table(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, cases)
    df_1 = pd.DataFrame(df_1, index=[0])
    df_list_1.append(df_1)
    cases = [2, 3]
    df_2 = bplt.generate_regression_table(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, cases)
    df_2 = pd.DataFrame(df_2, index=[0])
    df_list_2.append(df_2)

    

cases = [0, 1]
bplt.plot_1D_DeltaT_multiple(Qdot, SR_fp, caoMeshCode_list, testConditionsCode, method, no_try, cases).show()
cases = [2, 3]
bplt.plot_1D_DeltaT_multiple(Qdot, SR_fp, caoMeshCode_list, testConditionsCode, method, no_try, cases).show()

print("Résultats pour 0.5 m/s")
df_combined_1 = pd.concat(df_list_1, ignore_index=True)
pr.disp_html(df_combined_1)
print("Résultats pour 1.5 m/s")
df_combined_2 = pd.concat(df_list_2, ignore_index=True)
pr.disp_html(df_combined_2)

In [ ]:
SR_fp = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\SimulationResults'
# caoMeshCode = 'A5_M1'
caoMeshCode_list = ['A3_M1','A4_M1','A5_M1', 'A7_M1','A10_M2','A11_M1','A12_M1']
# caoMeshCode_list = ['A11_M1', 'A12_M1']
testConditionsCode = 'tC0'
method = 'bridge'
no_try =0
no_case = 1
cases = [0,1]
Qdot = 'Qdot_tube_fluid'
for no_case in cases :
    bplt.plot_1D_participation_stacked_last_it_multiple(SR_fp, caoMeshCode_list, testConditionsCode, method, no_try, no_case).show()
    # for caoMeshCode in caoMeshCode_list :
        # bplt.plot_1D_last_it_v2(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()

In [ ]:
SR_fp = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\SimulationResults'
caoMeshCode = 'A12_M1'
# caoMeshCode_list = ['A3_M1','A4_M1','A5_M1', 'A7_M1','A11_M1','A12_M1']
testConditionsCode = 'tC0'
method = 'bridge'
no_try =0
no_case = 0
cases = [0,1,2]
for no_case in cases :
    # plot_CFD_participation_last_it(SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()
    bplt.plot_1D_participation_last_it(SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()
    for Qdot in ['Qdot_tube_fluid', 'Qdot_top','Qdot_tube_back', 'Qdot_PV_sky', 'Qdot_top_rad', 'Qdot_top_conv', 'Qdot_tube_back_rad', 'Qdot_tube_back_conv'] :
        bplt.plot_big_it(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()

### Tests

In [ ]:
SR_fp =r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\SimulationResults'
caoMeshCodes = ['A5_M1','A7_M1','A8_M1','A10_M1']
cases = [0,1,2]
testConditionsCode = 'tC0'
method = 'bridge'
no_try =1
for no_case in cases :
    for caoMeshCode in caoMeshCodes:
        # plot_CFD_participation_last_it(SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()
        bplt.plot_1D_participation_last_it(SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()
        for Qdot in ['Qdot_tube_fluid', 'Qdot_top','Qdot_tube_back', 'Qdot_PV_sky', 'Qdot_top_rad', 'Qdot_top_conv', 'Qdot_tube_back_rad', 'Qdot_tube_back_conv'] :
            bplt.plot_big_it(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()

In [ ]:
SR_fp =r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\SimulationResults'
caoMeshCodes = ['A11_M1','A7_M1']
cases = [0,1,2,3]
testConditionsCode = 'tCwind'
method = 'bridge'
no_try =0
for no_case in cases :
    for caoMeshCode in caoMeshCodes:
        # plot_CFD_participation_last_it(SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()
        # bplt.plot_1D_participation_last_it(SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()
        for Qdot in ['Qdot_tube_fluid', 'Qdot_top','Qdot_tube_back', 'Qdot_PV_sky', 'Qdot_top_rad', 'Qdot_top_conv', 'Qdot_tube_back_rad', 'Qdot_tube_back_conv'] :
            bplt.plot_big_it(Qdot, SR_fp, caoMeshCode, testConditionsCode, method, no_try, no_case).show()